In [1]:
from mp_api.client import MPRester
from pymatgen.analysis.pourbaix_diagram import PourbaixDiagram, PourbaixPlotter, PourbaixEntry
from api_key import APIKEY

API_KEY = APIKEY

with MPRester(API_KEY) as mpr:
    docs = mpr.materials.summary.search(
        energy_above_hull=(0, 0.008), band_gap=[0.85,2.15], is_stable=True, fields=["material_id"]
    )
    materials = [doc.material_id for doc in docs]

Retrieving SummaryDoc documents:   0%|          | 0/5081 [00:00<?, ?it/s]

mp-866101


In [2]:
def get_material_entries(material_ids, batch_size=6000):
    mp_entries = []
    with MPRester(API_KEY) as mpr:
        for i in range(0, len(material_ids), batch_size):
            batch_ids = material_ids[i:i + batch_size]
            material_entries = mpr.materials.search(material_ids=batch_ids)
            mp_entries.extend(material_entries)
    return mp_entries

mp_entries = get_material_entries(materials)

Retrieving MaterialsDoc documents:   0%|          | 0/5081 [00:00<?, ?it/s]

In [3]:
chemsys_list = [entry.chemsys for entry in mp_entries[:30] if hasattr(entry, 'chemsys')]
composition_list = [entry.composition for entry in mp_entries[:30] if hasattr(entry, 'composition')]
materials_gga = [material + "-GGA" for material in materials[:30]]
materials_id = materials[:30]

In [11]:
print(materials_id)

[MPID(mp-866101), MPID(mp-861502), MPID(mp-568392), MPID(mp-560328), MPID(mp-861942), MPID(mp-690687), MPID(mp-558763), MPID(mp-1229122), MPID(mp-556866), MPID(mp-27966), MPID(mp-556429), MPID(mp-1190325), MPID(mp-707138), MPID(mp-13956), MPID(mp-36216), MPID(mp-5625), MPID(mp-16913), MPID(mp-29163), MPID(mp-561620), MPID(mp-662599), MPID(mp-12459), MPID(mp-555269), MPID(mp-1229155), MPID(mp-558550), MPID(mp-28812), MPID(mp-568540), MPID(mp-559847), MPID(mp-553919), MPID(mp-37153), MPID(mp-22665)]


In [4]:
from pymatgen.core.composition import Composition

def remove_o_and_h(composition):
    new_composition_elements = []
    for element, count in composition.get_el_amt_dict().items():
        if element not in ['O', 'H']:
            new_composition_elements.append(f"{element}{int(count)}")
    new_composition_str = ' '.join(new_composition_elements)
    return Composition(new_composition_str)

filtered_molecules = [remove_o_and_h(molecule) for molecule in composition_list]

In [5]:
from pymatgen.core.composition import Composition

def calculate_percent_composition(molecule):
    total_atoms = sum(molecule.values())
    return {element.symbol: count / total_atoms for element, count in molecule.items()}

percent_compositions = [calculate_percent_composition(molecule) for molecule in filtered_molecules]

In [6]:
import time
pbx_data = []
pbx_entries = []

total_iterations = len(chemsys_list)  

with MPRester(API_KEY) as mpr:
    count = 0  
    for i, x in zip(chemsys_list, percent_compositions):
        count += 1 
        try:
            time.sleep(2)
            pourbaix_entries = mpr.get_pourbaix_entries(i)
            time.sleep(2)
            pbx_entries.append(pourbaix_entries)
            pbx_diagram = PourbaixDiagram(entries=pourbaix_entries, comp_dict=x)
            pbx_data.append(pbx_diagram)
            time.sleep(2)
        except Exception as e:
            pbx_data.append('error')
        finally:
            percentage_complete = (count / total_iterations) * 100  
            print(f"Percentage complete: {percentage_complete:.2f}%")  



Retrieving ThermoDoc documents:   0%|          | 0/490 [00:00<?, ?it/s]

Percentage complete: 3.33%


Retrieving ThermoDoc documents:   0%|          | 0/530 [00:00<?, ?it/s]

Percentage complete: 6.67%


Retrieving ThermoDoc documents:   0%|          | 0/498 [00:00<?, ?it/s]

...

Percentage complete: 93.33%


Retrieving ThermoDoc documents:   0%|          | 0/282 [00:00<?, ?it/s]

Percentage complete: 96.67%


Retrieving ThermoDoc documents:   0%|          | 0/429 [00:00<?, ?it/s]

Percentage complete: 100.00%


In [7]:
matches = []

for index, comp in enumerate(composition_list):
    comp_formula = comp.formula
    found_match = False 

    try:
        for sublist in pbx_entries:
            if isinstance(sublist, list):
                for entry in sublist:
                    entry_str = str(entry)
                    if comp_formula and comp_formula in entry_str:
                        matches.append(entry)  
                        found_match = True
                        break

            if found_match:
                break

        if not found_match:
            print(f"No match found for: {comp_formula}")
            matches.append("None")

    except Exception as e:
        print(f"An error occurred at index {index} for composition {comp_formula}: {e}")
        matches.append("None")  


In [8]:
stabilities = []

for i, x in zip(pbx_data, matches):
    try:
        stability = i.get_decomposition_energy(x, 7, 0)
        formatted_stability = "{:.3g}".format(stability)
        stabilities.append(formatted_stability)
    except Exception as e:
        stabilities.append("error")


In [9]:
combined_list = list(zip(materials_id, stabilities))
data_without_error = [item for item in combined_list if item[1] != 'error']
filtered_data = [item for item in data_without_error if float(item[1]) <= 0.5]
mp_identifiers = [item[0] for item in filtered_data]

In [13]:
print("total materials:",len(materials_id))
print("materials without error:",len(data_without_error))
print("stable materials:",len(filtered_data))




# Extracting the IDs including 'mp-' but without 'MPID' and parentheses
extracted_ids = [mpid.split('(')[-1].rstrip(')') for mpid in mp_identifiers]
print(extracted_ids)

# The new list contains ['mp-568392', 'mp-560328', 'mp-558763', 'mp-27966', 'mp-556429', 'mp-1190325']



total materials: 30
materials without error: 30
stable materials: 14
['mp-568392', 'mp-560328', 'mp-558763', 'mp-27966', 'mp-556429', 'mp-1190325', 'mp-707138', 'mp-5625', 'mp-16913', 'mp-662599', 'mp-568540', 'mp-559847', 'mp-37153', 'mp-22665']
